# Embedding
It is a way to turn words into meaningful numbers. <br /> It stores how a word is used. It can capture things like which words appear near it, what it can replace, and what usually comes before or after it.

cat is close to dog pet and animal. <br />
food is close to drink, cook. <br /> <br />
These embeddings are learned automatically. <br /> <br />

`block_size` : How many tokens the model procress in one forward pass. <br />
In the example below we are taking 15 data in a chunk.


In [9]:
import string


data = string.printable

def get_chunk(block_size, idx):
    x = data[idx:idx + block_size]
    y = data[idx + 1:idx + block_size + 1]
    return x, y

(get_chunk(15, 2))

('23456789abcdefg', '3456789abcdefgh')

In [4]:
import torch.nn as nn


class Transformer(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.block_size = 25
        self.d_model = 128
        self.vocab_size = 75

        self.p_e = nn.Embedding(self.block_size, self.d_model)
        self.t_e = nn.Embedding(self.vocab_size, self.d_model)

transformer = Transformer()
print(transformer.p_e)
print(transformer.t_e)

Embedding(25, 128)
Embedding(75, 128)
